# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [14]:
driver = webdriver.Chrome()
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [15]:
#type in dirt
op_name = driver.find_element_by_name("OperSearch")
op_name.send_keys("dirt")

In [16]:
#check abandoned mines
#jk it's already checked
#abandon_check_box = driver.find_element_by_name("Abandoned")
#abandon_check_box.click()

In [17]:
#now search
search_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
search_button.click()

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

> - *Tip: `.text` will help you here.*
> - *Tip: You aren't interesting in annotations or anything, just mines and where they are from*
> - *Tip: Using `print("-----")` will help you keep track of different rows*
> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third. You can use this to skip ahead to the 'good' data if you want*

In [30]:
search_results = driver.find_elements_by_tag_name("tr")
for search_result in search_results[7:-1]:
    print(search_result.text)
    print("-------------------")

3503598
OR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC 
-------------------
1401575
KS  Bender Sand & Dirt   BENDER SAND & DIRT Surface M  Intermittent  Construction Sand and Gravel 
-------------------
5001797
AK  Dirt Company   Bush Pilot Surface M  Intermittent  Construction Sand and Gravel 
-------------------
2103723
MN  Dirt Doctor Inc   Rock Lake Plant Surface M  Intermittent  Construction Sand and Gravel 
-------------------
2103914
MN  Dirt Work Specialists LLC   Astec Plant Surface M  Intermittent  Construction Sand and Gravel 
-------------------
4104757
TX  Dirt Works   Portable #1 Surface M  Intermittent  Construction Sand and Gravel 
-------------------
0801306
FL  Holley Dirt Company, Inc   River Road Pit Surface M  Active  Sand, Common 
-------------------
3901432
SD  Krueger Brothers Gravel & Dirt   PORTABLE SCREENER Surface M  Intermittent  Construction Sand and Gravel 
-------------------
3609624
PA  M R Dirt   Forbes Pit S

### Loop through each operator result, printing its name

> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third.*
> - *Tip: You can use list slicing or an `if` statement to skip the non-data row(s). List slicing is probably easier, even if you aren't comfortable with it.*
> - *Tip: or honestly you can use `try` and `except` if you know how it works.*
> - *Tip: Once you have the "right" rows of data, you're going to be looking for a certain tag inside*
> - *Tip: Sometimes you can't say "give me this class," and instead you have to say "give me all of the `div` elements, and then give me the third one."*

In [34]:
search_results = driver.find_elements_by_tag_name("tr")

for search_result in search_results[7:-1]:
    tds = search_result.find_elements_by_tag_name("td")
    name = tds[2]
    print(name.text)

Newberg Rock & Dirt  
Bender Sand & Dirt  
Dirt Company  
Dirt Doctor Inc  
Dirt Work Specialists LLC  
Dirt Works  
Holley Dirt Company, Inc  
Krueger Brothers Gravel & Dirt  
M R Dirt  
M.R. Dirt Inc.  
P B Dirt Movers, Inc  
P B Dirt Movers, Inc.  
PB Dirt Movers  
Prescott Dirt, LLC  
R D Blankenship Dirt Work LLC  
Sand & Dirt, Inc  
SIMPSON DIRTWORX LLC  
SIMPSON DIRTWORX LLC  
Spry's Dirt & Gravel, Inc.  
Vogt Dirt Service  


### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [35]:
search_results = driver.find_elements_by_tag_name("tr")

for search_result in search_results[7:-1]:
    tds = search_result.find_elements_by_tag_name("td")
    id = tds[0]
    print(id.text)

3503598
1401575
5001797
2103723
2103914
4104757
0801306
3901432
3609624
3609931
1519799
4407379
4407296
0203332
2901986
0801417
4300768
4300776
2302283
2103518


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

> - *Tip: Start with an empty dictionary, then add the keys one at a time like we did during class*
> - *Tip: You might want to save all of the cells in a variable, then use indexes to get the second, third, fourth, etc.*
> - *Tip: I know you already skipped a bunch of rows already, but one of them still might be bad! Which one is it? How can you skip it? You might need to slice out some of the end of your list, too. Use `print` to help you debug, or just look at the page closely.*
> - *Tip: Or, if you did the other homework already, `try` / `except` is also an option*

In [47]:
search_results = driver.find_elements_by_tag_name("tr")
rows = []

for search_result in search_results:
    try:
        row = {}
        tds = search_result.find_elements_by_tag_name("td")
        
        #Operator ID
        id = tds[0]
        row["Operator ID"] = id.text
        
        #State
        state = tds[1]
        row["State"] = state.text
        
        #Operator name
        op_name = tds[2]
        row["Operator Name"] = op_name.text
        
        #Mine name
        mine_name = tds[3]
        row["Mine Name"] = mine_name.text
        
        #Mine type
        mine_type = tds[4]
        row["Mine Type"] = mine_type.text

        #Coal or metal
        cm = tds[5]
        row["Coal or Metal"] = cm.text

        #Status
        status = tds[6]
        row["Status"] = status.text
        
        #Commodity
        commodity = tds[7]
        row["Commodity"] = commodity.text
        
        rows.append(row)
    except:
        pass

print(rows)

###BUT! what if I want to create an ordered dictionary?

[{'Operator ID': '3503598', 'State': 'OR ', 'Operator Name': 'Newberg Rock & Dirt  ', 'Mine Name': 'Newberg Rock & Dirt', 'Mine Type': 'Surface', 'Coal or Metal': 'M ', 'Status': 'Active ', 'Commodity': 'Crushed, Broken Stone NEC '}, {'Operator ID': '1401575', 'State': 'KS ', 'Operator Name': 'Bender Sand & Dirt  ', 'Mine Name': 'BENDER SAND & DIRT', 'Mine Type': 'Surface', 'Coal or Metal': 'M ', 'Status': 'Intermittent ', 'Commodity': 'Construction Sand and Gravel '}, {'Operator ID': '5001797', 'State': 'AK ', 'Operator Name': 'Dirt Company  ', 'Mine Name': 'Bush Pilot', 'Mine Type': 'Surface', 'Coal or Metal': 'M ', 'Status': 'Intermittent ', 'Commodity': 'Construction Sand and Gravel '}, {'Operator ID': '2103723', 'State': 'MN ', 'Operator Name': 'Dirt Doctor Inc  ', 'Mine Name': 'Rock Lake Plant', 'Mine Type': 'Surface', 'Coal or Metal': 'M ', 'Status': 'Intermittent ', 'Commodity': 'Construction Sand and Gravel '}, {'Operator ID': '2103914', 'State': 'MN ', 'Operator Name': 'Dirt 

### Save that to a CSV named `dirt-operators.csv`

In [48]:
import pandas as pd

df = pd.DataFrame(rows)
df

,Coal or Metal,Commodity,Mine Name,Mine Type,Operator ID,Operator Name,State,Status
0,M,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Surface,3503598,Newberg Rock & Dirt,OR,Active
1,M,Construction Sand and Gravel,BENDER SAND & DIRT,Surface,1401575,Bender Sand & Dirt,KS,Intermittent
2,M,Construction Sand and Gravel,Bush Pilot,Surface,5001797,Dirt Company,AK,Intermittent
3,M,Construction Sand and Gravel,Rock Lake Plant,Surface,2103723,Dirt Doctor Inc,MN,Intermittent
4,M,Construction Sand and Gravel,Astec Plant,Surface,2103914,Dirt Work Specialists LLC,MN,Intermittent
5,M,Construction Sand and Gravel,Portable #1,Surface,4104757,Dirt Works,TX,Intermittent
6,M,"Sand, Common",River Road Pit,Surface,0801306,"Holley Dirt Company, Inc",FL,Active
7,M,Construction Sand and Gravel,PORTABLE SCREENER,Surface,3901432,Krueger Brothers Gravel & Dirt,SD,Intermittent
8,M,Construction Sand and Gravel,Forbes Pit,Surface,3609624,M R Dirt,PA,Temporarily Idled
9,M,Dimension Stone NEC,Camptown Quarry,Surface,3609931,M.R. Dirt Inc.,PA,Intermittent


In [50]:
df.shape

(20, 8)

In [51]:
df.to_csv("dirt-operators.csv", index = False)

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.

In [52]:
df1 = pd.read_csv("dirt-operators.csv")
df1

,Coal or Metal,Commodity,Mine Name,Mine Type,Operator ID,Operator Name,State,Status
0,M,"Crushed, Broken Stone NEC",Newberg Rock & Dirt,Surface,3503598,Newberg Rock & Dirt,OR,Active
1,M,Construction Sand and Gravel,BENDER SAND & DIRT,Surface,1401575,Bender Sand & Dirt,KS,Intermittent
2,M,Construction Sand and Gravel,Bush Pilot,Surface,5001797,Dirt Company,AK,Intermittent
3,M,Construction Sand and Gravel,Rock Lake Plant,Surface,2103723,Dirt Doctor Inc,MN,Intermittent
4,M,Construction Sand and Gravel,Astec Plant,Surface,2103914,Dirt Work Specialists LLC,MN,Intermittent
5,M,Construction Sand and Gravel,Portable #1,Surface,4104757,Dirt Works,TX,Intermittent
6,M,"Sand, Common",River Road Pit,Surface,801306,"Holley Dirt Company, Inc",FL,Active
7,M,Construction Sand and Gravel,PORTABLE SCREENER,Surface,3901432,Krueger Brothers Gravel & Dirt,SD,Intermittent
8,M,Construction Sand and Gravel,Forbes Pit,Surface,3609624,M R Dirt,PA,Temporarily Idled
9,M,Dimension Stone NEC,Camptown Quarry,Surface,3609931,M.R. Dirt Inc.,PA,Intermittent
